## Import Dependencies

In [1]:
#import the dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
from datetime import datetime
from citipy import citipy

#import API Key
from config import weather_api_key
from config import g_key
#Output file CSV


## Generate Cities List

In [2]:
#create lists to hold lat_lng  and cities
lat_lngs = []
cities = []
countries = []

#create a set of random latitude and longitude combinations (2000)
lats = np.random.uniform(-90, 90, size=2000)
lngs = np.random.uniform(-180, 180, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

#identify the nearest city for each latitude and longitude combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    country = citipy.nearest_city(lat_lng[0], lat_lng[1]).country_code
    #if city is unique, add to list of cities
    if city not in cities:
        cities.append(city)
        countries.append(country)
#print the city and country count to confirm sufficient count
print(len(cities))
print(len(countries))

744
744


In [3]:
#create a dataframe to store data
city_weather = pd.DataFrame({
    'City': cities,
    'Cloudiness': "",
    'Country': countries,
    'Date': "",
    'Humidity': "",
    'Latitude': "",
    'Longitude': "",
    'Max Temp': "",
    'Windspeed': ""})
#view first 5 
city_weather.head()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Windspeed
0,rikitea,,pf,,,,,,
1,bluff,,nz,,,,,,
2,albany,,au,,,,,,
3,arman,,ru,,,,,,
4,mglin,,ru,,,,,,


## API Calls


In [4]:
##general a url for querying openweathermap
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=b54ae86f918e06974cda27d428917778


In [5]:
##prepare empty city list for processing the import
# start with an empty list for weather data
city_data=[]  # empty list

#print the beginning of the logging.
print('''----------------------------- \n Beginning Data Retrieval \n -----------------------------''')

#create counters.
record_count = 1  # number of records in crawl
set_count = 1  # # of sets in crawl.  50 cities per set.

#loop through all the cities in the list.
for i, city in enumerate(cities): # for loop with enumerate. reference index and city var

    #group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50): ## if index divisble by 50 and index> 50, creates a new group
        set_count += 1  # adds one to set count 
        record_count = 1 # resets record count to 1
    #create endpoint URL with each city.
    city_url = url + "&q=" + city

    #log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    #add 1 to the record count.
    record_count += 1
    #run an API request for each of the cities.
    try:
        #parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        #parse out the needed data.
        city_country = city_weather["sys"]["country"]
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        
        ##challenge Weather Description
        city_weather_main=city_weather["weather"][0]["main"]   
        city_weather_desc = city_weather["weather"][0]["description"]
        
        ## challenge - get inches of rain in last three hours where it has rained.
        try:
            if city_weather["weather"][0]["main"]=="Rain":
                city_rain_inches= city_weather["rain"]["3h"]
            else: 
                city_rain_inches=0
        except:  print (f"failing on the rain. skipping")
             
        ##challenge - except practice.  Now do with snow
        try:
            if city_weather["weather"][0]["main"]=="Snow":
                city_snow_inches= city_weather["snow"]["3h"]
            else: 
                city_snow_inches=0
        except:  print (f"failing on the snow. skipping")
              
        
        #convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
            #append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Date": city_date,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Overall Weather" : city_weather_main,
                          "Weather Description" : city_weather_desc,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Rain Inches (last 3 hrs)" : city_rain_inches,
                          "Snow Inches (last 3 hrs)" : city_snow_inches
                          })
         
                         
#if an error is experienced, skip the city.
    except:
        print("City not found... Skipping.")
        pass  ## general purpose statement to handle all errors and continue

#indicate that Data Loading is complete.
print('''----------------------------- \n Data Retrieval Complete \n -----------------------------''')

----------------------------- 
 Beginning Data Retrieval 
 -----------------------------
Processing Record 1 of Set 1 | rikitea
Processing Record 2 of Set 1 | bluff
Processing Record 3 of Set 1 | albany
Processing Record 4 of Set 1 | arman
Processing Record 5 of Set 1 | mglin
Processing Record 6 of Set 1 | los llanos de aridane
Processing Record 7 of Set 1 | nizwa
Processing Record 8 of Set 1 | carnarvon
Processing Record 9 of Set 1 | saiha
failing on the rain. skipping
Processing Record 10 of Set 1 | kodiak
failing on the rain. skipping
Processing Record 11 of Set 1 | vaitupu
City not found... Skipping.
Processing Record 12 of Set 1 | beringovskiy
Processing Record 13 of Set 1 | barrow
Processing Record 14 of Set 1 | chokurdakh
Processing Record 15 of Set 1 | punta arenas
Processing Record 16 of Set 1 | saskylakh
failing on the rain. skipping
Processing Record 17 of Set 1 | busselton
Processing Record 18 of Set 1 | ushuaia
Processing Record 19 of Set 1 | lebu
Processing Record 20 of S

Processing Record 25 of Set 4 | port hedland
Processing Record 26 of Set 4 | grand gaube
Processing Record 27 of Set 4 | lompoc
Processing Record 28 of Set 4 | sitka
Processing Record 29 of Set 4 | kahului
Processing Record 30 of Set 4 | monticello
Processing Record 31 of Set 4 | huainan
failing on the rain. skipping
Processing Record 32 of Set 4 | olinda
Processing Record 33 of Set 4 | taoudenni
Processing Record 34 of Set 4 | cabo san lucas
Processing Record 35 of Set 4 | isangel
Processing Record 36 of Set 4 | oistins
Processing Record 37 of Set 4 | guangyuan
Processing Record 38 of Set 4 | banda aceh
Processing Record 39 of Set 4 | parksville
Processing Record 40 of Set 4 | athabasca
Processing Record 41 of Set 4 | calbuco
failing on the rain. skipping
Processing Record 42 of Set 4 | lata
Processing Record 43 of Set 4 | halmaj
Processing Record 44 of Set 4 | bensonville
failing on the rain. skipping
Processing Record 45 of Set 4 | parry sound
Processing Record 46 of Set 4 | rocha
P

Processing Record 7 of Set 8 | nouakchott
Processing Record 8 of Set 8 | kaberamaido
Processing Record 9 of Set 8 | namibe
Processing Record 10 of Set 8 | vardo
Processing Record 11 of Set 8 | saint-joseph
Processing Record 12 of Set 8 | teahupoo
Processing Record 13 of Set 8 | dong xoai
Processing Record 14 of Set 8 | hinesville
Processing Record 15 of Set 8 | manzil jamil
City not found... Skipping.
Processing Record 16 of Set 8 | ostersund
Processing Record 17 of Set 8 | monroe
Processing Record 18 of Set 8 | ruteng
Processing Record 19 of Set 8 | mount gambier
Processing Record 20 of Set 8 | guerrero negro
Processing Record 21 of Set 8 | kirakira
Processing Record 22 of Set 8 | lalmohan
Processing Record 23 of Set 8 | flinders
Processing Record 24 of Set 8 | nyaunglebin
Processing Record 25 of Set 8 | itaituba
Processing Record 26 of Set 8 | dali
Processing Record 27 of Set 8 | talnakh
Processing Record 28 of Set 8 | haines junction
Processing Record 29 of Set 8 | sataua
City not f

Processing Record 33 of Set 11 | zamora
failing on the rain. skipping
Processing Record 34 of Set 11 | mizdah
Processing Record 35 of Set 11 | pinheiro machado
Processing Record 36 of Set 11 | winsum
Processing Record 37 of Set 11 | guilin
Processing Record 38 of Set 11 | galveston
Processing Record 39 of Set 11 | seoul
failing on the rain. skipping
Processing Record 40 of Set 11 | kavaratti
Processing Record 41 of Set 11 | taunggyi
failing on the rain. skipping
Processing Record 42 of Set 11 | nang rong
failing on the rain. skipping
Processing Record 43 of Set 11 | valparaiso
Processing Record 44 of Set 11 | leninskoye
Processing Record 45 of Set 11 | ucluelet
Processing Record 46 of Set 11 | brigantine
Processing Record 47 of Set 11 | matagami
Processing Record 48 of Set 11 | palimbang
Processing Record 49 of Set 11 | quatre cocos
failing on the rain. skipping
Processing Record 50 of Set 11 | khonuu
City not found... Skipping.
Processing Record 1 of Set 12 | bone
Processing Record 2 

Processing Record 8 of Set 15 | phalombe
Processing Record 9 of Set 15 | wadi maliz
Processing Record 10 of Set 15 | sinnamary
Processing Record 11 of Set 15 | newala
Processing Record 12 of Set 15 | adre
Processing Record 13 of Set 15 | fort nelson
Processing Record 14 of Set 15 | tshane
Processing Record 15 of Set 15 | douglas
Processing Record 16 of Set 15 | kyren
Processing Record 17 of Set 15 | skibbereen
Processing Record 18 of Set 15 | oshogbo
failing on the rain. skipping
Processing Record 19 of Set 15 | bouar
Processing Record 20 of Set 15 | garden city
failing on the rain. skipping
Processing Record 21 of Set 15 | impfondo
Processing Record 22 of Set 15 | atocha
Processing Record 23 of Set 15 | east linton
Processing Record 24 of Set 15 | rundu
Processing Record 25 of Set 15 | guarapari
Processing Record 26 of Set 15 | manicore
Processing Record 27 of Set 15 | gurgan
City not found... Skipping.
Processing Record 28 of Set 15 | domoni
Processing Record 29 of Set 15 | chapais
P

In [6]:
###add the collected data to a new dataframe.
city_data_df=pd.DataFrame(city_data)
city_data_df.head(5)

,City,Country,Date,Lat,Lng,Overall Weather,Weather Description,Max Temp,Humidity,Cloudiness,Wind Speed,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
0,Rikitea,PF,2021-06-25 15:23:06,-23.1203,-134.9692,Clouds,few clouds,74.03,77,14,3.60,0,0
1,Bluff,NZ,2021-06-25 15:23:07,-46.6000,168.3333,Clouds,overcast clouds,48.79,91,90,6.33,0,0
2,Albany,US,2021-06-25 15:22:40,42.6001,-73.9662,Clouds,overcast clouds,75.61,54,100,4.47,0,0
3,Arman,RU,2021-06-25 15:23:07,59.7000,150.1667,Clouds,scattered clouds,48.45,88,45,2.17,0,0
4,Mglin,RU,2021-06-25 15:14:51,53.0591,32.8475,Clear,clear sky,87.78,61,9,4.52,0,0


In [7]:
#save df to csv file
output_data_file = "Weather_Database/WeatherPy_Database.csv"
city_data_df.to_csv(output_data_file, index_label='City_ID')